In [90]:

from torch.optim import Adam

from torchkge.evaluation import LinkPredictionEvaluator
from torchkge.models import TransEModel
from torchkge.utils.datasets import load_fb15k
from torchkge.utils import Trainer, MarginLoss, DataLoader
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils.pretrained_models import load_pretrained_transe



from tqdm.autonotebook import tqdm

In [85]:
import torch


In [86]:
from torchkge.evaluation import TripletClassificationEvaluator

In [87]:
from rdflib import Graph
import pandas as pd
from torchkge.data_structures import KnowledgeGraph

<div class="alert alert-block alert-info" style="color: #01571B; background-color: #C8E5C6;">


From the model prepared Turtle file we build the desired graph:

In [61]:


# Load the graph from TTL file
g = Graph()
g.parse("./../../data/explotation_zone/RDFGraph_Model_emb.ttl", format="ttl")


triples = []
for s, p, o in g:
    triples.append((str(s), str(p), str(o)))

data = pd.DataFrame(triples)
data.columns = ['from','rel','to']
data = data[['from','to','rel']]



In [62]:
data[data['rel'] == 'http://example.org/apartment/price_discretized']

,from,to,rel
25,http://example.org/apartment/apartment_4198,1.0,http://example.org/apartment/price_discretized
65,http://example.org/apartment/apartment_8904,1.0,http://example.org/apartment/price_discretized
66,http://example.org/apartment/apartment_9032,0.0,http://example.org/apartment/price_discretized
83,http://example.org/apartment/apartment_7323,1.0,http://example.org/apartment/price_discretized
93,http://example.org/apartment/apartment_7506,0.0,http://example.org/apartment/price_discretized
...,...,...,...
288210,http://example.org/apartment/apartment_8324,0.0,http://example.org/apartment/price_discretized
288323,http://example.org/apartment/apartment_8343,1.0,http://example.org/apartment/price_discretized
288365,http://example.org/apartment/apartment_8381,3.0,http://example.org/apartment/price_discretized
288373,http://example.org/apartment/apartment_3097,1.0,http://example.org/apartment/price_discretized


<div class="alert alert-block alert-info" style="color: #01571B; background-color: #C8E5C6;">

We split the data between train and test. Since we only want to predict the price of AirBNB apartments, only triples corresponding to the price relation are going to be evaluated. The model will still learn the relations between all nodes, including the nodes for which the later model will be predicting it's price group (although this relations has been erased)

In [65]:
test = data[data['rel'] == 'http://example.org/apartment/price_discretized'].sample(frac = 0.2)
train = data.drop(test.index)
test_y = test['to']
test_X = test.drop(columns = ['to'])

Let's build the training graph

In [73]:
kg_train = KnowledgeGraph(df=train)


The model will be giving an output of 100 dimension embedding and will be trained with the parameters that can be seen in the code. The selected model is a TransE projection model. This selection has been made due to it's simplicity and therefore faster training.

In [74]:
emb_dim = 100
model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')

In [75]:
lr = 0.0004
margin = 0.5
n_epochs = 1000
b_size = 32768

In [76]:
optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
criterion = MarginLoss(margin)
sampler = BernoulliNegativeSampler(kg_train)
dataloader = DataLoader(kg_train, batch_size=b_size, use_cuda=None)

iterator = tqdm(range(n_epochs), unit='epoch')
c = 0
for epoch in iterator:
    print(f'epoch done! {c}')
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        h, t, r = batch[0], batch[1], batch[2]
        n_h, n_t = sampler.corrupt_batch(h, t, r)

        optimizer.zero_grad()

        # forward + backward + optimize
        pos, neg = model(h, t, r, n_h, n_t)
        loss = criterion(pos, neg)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    iterator.set_description(
        'Epoch {} | mean loss: {:.5f}'.format(epoch + 1,
                                              running_loss / len(dataloader)))
    c += 1

model.normalize_parameters()

  0%|          | 0/1000 [00:00<?, ?epoch/s]

epoch done! 0


Epoch 1 | mean loss: 15572.52072:   0%|          | 1/1000 [00:01<21:38,  1.30s/epoch]

epoch done! 1


Epoch 2 | mean loss: 13641.67893:   0%|          | 2/1000 [00:02<21:07,  1.27s/epoch]

...

Epoch 998 | mean loss: 8.24851: 100%|█████████▉| 998/1000 [20:37<00:02,  1.23s/epoch]

epoch done! 998


Epoch 999 | mean loss: 7.63405: 100%|█████████▉| 999/1000 [20:38<00:01,  1.24s/epoch]

epoch done! 999


Epoch 1000 | mean loss: 8.81968: 100%|██████████| 1000/1000 [20:40<00:00,  1.24s/epoch]


In [80]:
import pickle

The following cell is used as a mere checkpoint for the notebook

In [81]:
'''
with open('train.pkl', 'wb') as outp:
    pickle.dump(train, outp, pickle.HIGHEST_PROTOCOL)

with open('kg_train.pkl', 'wb') as outp:
    pickle.dump(kg_train, outp, pickle.HIGHEST_PROTOCOL)

with open('test_X.pkl', 'wb') as outp:
    pickle.dump(test_X, outp, pickle.HIGHEST_PROTOCOL)

with open('test_y.pkl', 'wb') as outp:
    pickle.dump(test_y, outp, pickle.HIGHEST_PROTOCOL)

with open('model.pkl', 'wb') as outp:
    pickle.dump(model, outp, pickle.HIGHEST_PROTOCOL)'''

In [89]:
with open('train.pkl', 'rb') as outp:
    train = pickle.load(outp)

with open('kg_train.pkl', 'rb') as outp:
    kg_train = pickle.load(outp)

with open('test_X.pkl', 'rb') as outp:
    test_X = pickle.load(outp)

with open('test_y.pkl', 'rb') as outp:
    test_y = pickle.load(outp)

with open('model.pkl', 'rb') as outp:
    model = pickle.load(outp)


In [92]:
test_X

,from,rel
29095,http://example.org/apartment/apartment_7211,http://example.org/apartment/price_discretized
203629,http://example.org/apartment/apartment_108,http://example.org/apartment/price_discretized
263343,http://example.org/apartment/apartment_4408,http://example.org/apartment/price_discretized
147769,http://example.org/apartment/apartment_6169,http://example.org/apartment/price_discretized
224995,http://example.org/apartment/apartment_5027,http://example.org/apartment/price_discretized
...,...,...
64726,http://example.org/apartment/apartment_4196,http://example.org/apartment/price_discretized
280219,http://example.org/apartment/apartment_267,http://example.org/apartment/price_discretized
131051,http://example.org/apartment/apartment_2304,http://example.org/apartment/price_discretized
87592,http://example.org/apartment/apartment_8099,http://example.org/apartment/price_discretized


<div class="alert alert-block alert-info" style="color: #01571B; background-color: #C8E5C6;">

Now, and using batch-processing to not overload the memory, we will get the embedding for each apartment in the training and test datasets separately. This will be later loaded in a torch tensor to train a future network.

Train

In [83]:
import numpy as np

train_embedding_list = []
batch_size = 1000  # Adjust batch size according to your memory capacity

filtered_items = train[train['rel'] == 'http://example.org/apartment/price_discretized']['from']

for start in range(0, len(filtered_items), batch_size):
    batch = filtered_items[start:start + batch_size]
    batch_embeddings = []
    for item in batch:
        i = kg_train.ent2ix[item]
        embedding = model.get_embeddings()[0][i]
        if isinstance(embedding, np.ndarray):
            batch_embeddings.append(embedding)
        else:
            batch_embeddings.append(np.array(embedding))  # Ensure it is a NumPy array
    
    # Convert batch_embeddings to a NumPy array
    batch_embeddings = np.stack(batch_embeddings)
    
    # Save intermediate results to disk
    np.save(f'batch_{start}.npy', batch_embeddings)



Test

In [96]:

train_embedding_list = []
batch_size = 500  # Adjust batch size according to your memory capacity

filtered_items = test_X['from']

for start in range(0, len(filtered_items), batch_size):
    batch = filtered_items[start:start + batch_size]
    batch_embeddings = []
    for item in batch:
        i = kg_train.ent2ix[item]
        embedding = model.get_embeddings()[0][i]
        if isinstance(embedding, np.ndarray):
            batch_embeddings.append(embedding)
        else:
            batch_embeddings.append(np.array(embedding))  # Ensure it is a NumPy array
    
    # Convert batch_embeddings to a NumPy array
    batch_embeddings = np.stack(batch_embeddings)
    
    # Save intermediate results to disk
    np.save(f'./test_X/batch_{start}.npy', batch_embeddings)

